In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sqlite3
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("/Users/thomassimmons/c/d/datasets/dirty_cafe_sales.csv")

In [5]:
df.shape

(10000, 8)

In [6]:
df['Quantity'] = pd.to_numeric(df['Quantity'], errors = 'coerce')

In [10]:
df.dtypes

Transaction ID       object
Item                 object
Quantity            float64
Price Per Unit      float64
Total Spent         float64
Payment Method       object
Location             object
Transaction Date     object
dtype: object

In [8]:
df['Price Per Unit'] = pd.to_numeric(df['Price Per Unit'], errors = 'coerce')

In [9]:
df['Total Spent'] = pd.to_numeric(df['Total Spent'], errors = 'coerce')

In [11]:
df['Payment Method'] = df['Payment Method'].replace({'UNKNOWN': pd.NA})

In [12]:
df['Location'] = df['Location'].replace({'UNKNOWN': pd.NA})

In [14]:
df.dropna(subset=['Quantity', 'Price Per Unit', 'Total Spent', 'Payment Method', 'Location'], inplace=True)

In [16]:
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], errors = 'coerce')

In [17]:
df.dropna(subset=['Transaction Date'], inplace=True)

In [ ]:
df.reset_index(drop=True, inplace=True)